In [1]:
%cd ..\src
!python setup.py develop

c:\Users\mscer\dev\EchoVPR\src
running develop
running egg_info
writing echovpr.egg-info\PKG-INFO
writing dependency_links to echovpr.egg-info\dependency_links.txt
writing top-level names to echovpr.egg-info\top_level.txt
reading manifest file 'echovpr.egg-info\SOURCES.txt'
writing manifest file 'echovpr.egg-info\SOURCES.txt'
running build_ext
Creating c:\users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\echovpr.egg-link (link to .)
echovpr 1.0 is already the active version in easy-install.pth

Installed c:\users\mscer\dev\echovpr\src
Processing dependencies for echovpr==1.0
Finished processing dependencies for echovpr==1.0


C:\Users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\setuptools\command\easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
C:\Users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


In [2]:
from os.path import isfile, join

import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
from torchmetrics import MetricCollection

from configs.utils import get_config, get_config_full, get_int_from_config, get_float_from_config, get_bool_from_config
from echovpr.datasets.utils import get_dataset, get_subset_dataset, save_tensor
from echovpr.datasets.image_ds import ImageDataset
from echovpr.models.utils import get_sparsity
from echovpr.models.single_esn import SingleESN
from echovpr.models.hier_esn import HierESN
from echovpr.models.sparce_layer import SpaRCe
from echovpr.trainer.metrics.recall_top_k_metric import RecallTopKMetric

import logging

logging.basicConfig(level=logging.INFO)

In [3]:
config = get_config("configs\\train_esn_nordland_full_sweep.ini", log=False)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
run = wandb.init()
artifact = run.use_artifact('uos_ml/echovpr/esn_9km0ic3z:v0', type='model')
artifact_dir = artifact.download()

model_file = join(artifact_dir, 'model.pt')
esn_model_file = join(artifact_dir, 'esn_model.pt')

all_in_one = not isfile(esn_model_file)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mscerri (use `wandb login --relogin` to force relogin)


wandb: Downloading large artifact esn_9km0ic3z:v0, 1101.55MB. 2 files... Done. 0:0:0


In [6]:
in_features=int(config['model_in_features'])
reservoir_size=int(config['model_reservoir_size'])
out_features=int(config['model_out_features'])

esn_alpha = float(config['model_esn_alpha'])
esn_gamma = float(config['model_esn_gamma'])
esn_rho = float(config['model_esn_rho'])
esn_num_connections = int(config['model_esn_num_connections'])
sparce_enabled = get_bool_from_config(config, 'model_sparce_enabled')

model = nn.ModuleDict()

esn_model = SingleESN(
  in_features, 
  reservoir_size, 
  alpha=esn_alpha, 
  gamma=esn_gamma, 
  rho=esn_rho,
  sparsity=get_sparsity(esn_num_connections, reservoir_size),
  device=device
)

if all_in_one:
  model["esn"] = esn_model

if sparce_enabled:
  model["sparce"] = SpaRCe(reservoir_size)

model["out"] = nn.Linear(in_features=reservoir_size, out_features=out_features, bias=True)

In [7]:
if not all_in_one:
  esn_model.load_state_dict(torch.load(esn_model_file))

model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [8]:
if not all_in_one:
  esn_model.eval().to(device)
  
model.eval().to(device)

ModuleDict(
  (out): Linear(in_features=8000, out_features=27592, bias=True)
)

In [9]:
summer_dataset = get_dataset(config['dataset_nordland_summer_hidden_repr_file_path'])
winter_dataset = get_dataset(config['dataset_nordland_winter_hidden_repr_file_path'])

max_n = summer_dataset.tensors[0].max()
_ = summer_dataset.tensors[0].divide_(max_n)
_ = winter_dataset.tensors[0].divide_(max_n)

In [10]:
def process(model, dataLoader, device: torch.device):
    x_processed_list = []
    y_target_list = []
    
    for x, y_target in dataLoader:
        x = x.to(device)
        x_processed = model(x)

        x_processed_list.append(x_processed.cpu())
        y_target_list.append(y_target)

    return (torch.vstack(x_processed_list), torch.vstack(y_target_list))

In [11]:
print(f"Winter dataset size: {len(winter_dataset)}")
winter_dataLoader = DataLoader(winter_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

Winter dataset size: 27592


In [12]:
winter_dataset = TensorDataset(*process(esn_model, winter_dataLoader, device))

In [13]:
# Prepare Datasets

val_dataset = get_subset_dataset(winter_dataset, config['dataset_nordland_winter_val_limit_indices_file_path'])
print(f"Validation dataset size: {len(val_dataset)}")
val_dataLoader = DataLoader(val_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

test_dataset = get_subset_dataset(winter_dataset, config['dataset_nordland_winter_test_limit_indices_file_path'])
print(f"Test dataset size: {len(test_dataset)}")
test_dataLoader = DataLoader(test_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

Validation dataset size: 2759
Test dataset size: 24833


In [14]:
val_dataset_quantiles = None

if sparce_enabled:
    # Calculate Training Dataset Quantiles
    quantile = float(config['model_sparce_quantile'])
    val_dataset_quantiles = torch.quantile(torch.abs(torch.vstack([t[0] for t in val_dataset])), quantile, dim=0).to(device)

In [15]:
def eval_esn(model, dataLoader, sparce_enabled, quantiles, top_k = 100):
    predictions = []
    ground_truths = []

    with torch.no_grad():    
        for x, y_target in dataLoader:

            x = x.to(device)
            
            if sparce_enabled:
                x = model["sparce"](x, quantiles)

            preds = model["out"](x)

            _, predIdx = torch.topk(preds, top_k, dim=1)

            predictions.append(predIdx.cpu())
            ground_truths.append(torch.argmax(y_target, dim=1, keepdim=True))

    return (torch.vstack(predictions), torch.vstack(ground_truths))

In [16]:
val_predictions = TensorDataset(*eval_esn(model, val_dataLoader, sparce_enabled, val_dataset_quantiles))
test_predictions = TensorDataset(*eval_esn(model, test_dataLoader, sparce_enabled, val_dataset_quantiles))

In [17]:
# memory cleanup
del model
del val_dataset
del val_dataLoader
del test_dataset
del test_dataLoader
del winter_dataset
del winter_dataLoader

torch.cuda.empty_cache()

In [18]:
patchnetvlad_config = get_config_full("echovpr\\configs\\eval_patchnetvlad.ini")
patchnetvlad_main_config = patchnetvlad_config['main']

In [19]:
index_dataset = ImageDataset(patchnetvlad_main_config['dataset_nordland_summer_file_path'], patchnetvlad_main_config['dataset_root_dir'], patchnetvlad_main_config)
query_dataset = ImageDataset(patchnetvlad_main_config['dataset_nordland_winter_file_path'], patchnetvlad_main_config['dataset_root_dir'], patchnetvlad_main_config)

Parsing dataset...
Done! Found 27592 images
Final dataset size, 27592 images
Parsing dataset...
Done! Found 27592 images
Final dataset size, 27592 images


In [20]:
import numpy as np
from echovpr.models.netvlad_encoder import NetVLADEncorder
from patchnetvlad.tools.local_matcher import calc_keypoint_centers_from_patches, normalise_func
from patchnetvlad.tools.patch_matcher import PatchMatcher

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


In [21]:
# encoder = NetVLADEncorder(patchnetvlad_main_config).eval().to(device)

In [22]:
# def local_matcher_online(predictions, config, device):
#     patch_sizes = [int(s) for s in config['main']['model_patch_sizes'].split(",")]
#     strides = [int(s) for s in config['main']['model_strides'].split(",")]
#     patch_weights = np.array(config['feature_match']['patchWeights2Use'].split(",")).astype(float)

#     all_keypoints = []
#     all_indices = []

#     for patch_size, stride in zip(patch_sizes, strides):
#         # we currently only provide support for square patches, but this can be easily modified for future works
#         keypoints, indices = calc_keypoint_centers_from_patches(config['feature_match'], patch_size, patch_size, stride, stride)
#         all_keypoints.append(keypoints)
#         all_indices.append(indices)

#     reordered_preds = []

#     matcher = PatchMatcher(config['feature_match']['matcher'], patch_sizes, strides, all_keypoints, all_indices)

#     total_preds = len(predictions)

#     for i, (pred, q_idx) in enumerate(predictions):
#         if i % 10:
#             print(f"Processing query {i+1}/{total_preds}")
            
#         diffs = np.zeros((pred.shape[0], len(patch_sizes)))
        
#         qImg, _ = query_dataset[q_idx]
#         _, qfeat = encoder(qImg.unsqueeze(0).to(device))
#         qfeat = [torch.transpose(f[1].squeeze(0), 0, 1) for f in qfeat]

#         for k, candidate in enumerate(pred):
#             dbImg, _ = index_dataset[candidate]
#             _, dbfeat = encoder(dbImg.unsqueeze(0).to(device))
#             dbfeat = [f[1].squeeze(0) for f in dbfeat]
            
#             diffs[k, :], _, _ = matcher.match(qfeat, dbfeat)

#         diffs = normalise_func(diffs, len(patch_sizes), patch_weights)
#         cand_sorted = np.argsort(diffs)
#         reordered_preds.append(pred[cand_sorted])
    
#     return reordered_preds

In [23]:
import os

from joblib import Parallel, delayed, parallel_backend

In [24]:
index_input_features_dir = "C:\\Users\\mscer\\dev\\Patch-NetVLAD\\patchnetvlad\\output_features\\nordland_index"
query_input_features_dir = "C:\\Users\\mscer\\dev\\Patch-NetVLAD\\patchnetvlad\\output_features\\nordland_query"
input_query_local_features_prefix = join(query_input_features_dir, 'patchfeats')
input_query_global_features_prefix = join(query_input_features_dir, 'globalfeats.npy')
input_index_local_features_prefix = join(index_input_features_dir, 'patchfeats')
input_index_global_features_prefix = join(index_input_features_dir, 'globalfeats.npy')

def local_matcher_loop(item, total_preds,patch_sizes, matcher, patch_weights, input_query_local_features_prefix, input_index_local_features_prefix, device):
    i, (pred, q_idx) = item

    # if i % 1000 == 0:
    print(f"Processing query {i+1}/{total_preds}")

    diffs = np.zeros((pred.shape[0], len(patch_sizes)))
        
    image_name_query = os.path.splitext(os.path.basename(query_dataset.images[q_idx]))[0]
    qfeat = []
    for patch_size in patch_sizes:
        qfilename = input_query_local_features_prefix + '_' + 'psize{}_'.format(patch_size) + image_name_query + '.npy'
        qfeat.append(torch.transpose(torch.tensor(np.load(qfilename), device=device), 0, 1))
        # we pre-transpose here to save compute speed
        
    for k, candidate in enumerate(pred):
        image_name_index = os.path.splitext(os.path.basename(index_dataset.images[candidate]))[0]
        dbfeat = []
        for patch_size in patch_sizes:
            dbfilename = input_index_local_features_prefix + '_' + 'psize{}_'.format(patch_size) + image_name_index + '.npy'
            dbfeat.append(torch.tensor(np.load(dbfilename), device=device))

        diffs[k, :], _, _ = matcher.match(qfeat, dbfeat)

    diffs = normalise_func(diffs, len(patch_sizes), patch_weights)
    cand_sorted = np.argsort(diffs)

    return pred[cand_sorted]

def local_matcher(predictions, config, device):
    patch_sizes = [int(s) for s in config['main']['model_patch_sizes'].split(",")]
    strides = [int(s) for s in config['main']['model_strides'].split(",")]
    patch_weights = np.array(config['feature_match']['patchWeights2Use'].split(",")).astype(float)

    all_keypoints = []
    all_indices = []

    for patch_size, stride in zip(patch_sizes, strides):
        # we currently only provide support for square patches, but this can be easily modified for future works
        keypoints, indices = calc_keypoint_centers_from_patches(config['feature_match'], patch_size, patch_size, stride, stride)
        all_keypoints.append(keypoints)
        all_indices.append(indices)

    reordered_preds = []

    matcher = PatchMatcher(config['feature_match']['matcher'], patch_sizes, strides, all_keypoints, all_indices)

    total_preds = len(predictions)

    reordered_preds = Parallel(n_jobs=20, prefer="threads", require='sharedmem')(delayed(local_matcher_loop)(item, total_preds,patch_sizes, matcher, patch_weights, input_query_local_features_prefix, input_index_local_features_prefix, device) for item in enumerate(predictions))
    
    # for i, (pred, q_idx) in enumerate(predictions):
    #     if i % 1000 == 0:
    #         print(f"Processing query {i+1}/{total_preds}")
            
    #     diffs = np.zeros((pred.shape[0], len(patch_sizes)))
        
    #     image_name_query = os.path.splitext(os.path.basename(query_dataset.images[q_idx]))[0]
    #     qfeat = []
    #     for patch_size in patch_sizes:
    #         qfilename = input_query_local_features_prefix + '_' + 'psize{}_'.format(patch_size) + image_name_query + '.npy'
    #         qfeat.append(torch.transpose(torch.tensor(np.load(qfilename), device=device), 0, 1))
    #         # we pre-transpose here to save compute speed
            
    #     for k, candidate in enumerate(pred):
    #         image_name_index = os.path.splitext(os.path.basename(index_dataset.images[candidate]))[0]
    #         dbfeat = []
    #         for patch_size in patch_sizes:
    #             dbfilename = input_index_local_features_prefix + '_' + 'psize{}_'.format(patch_size) + image_name_index + '.npy'
    #             dbfeat.append(torch.tensor(np.load(dbfilename), device=device))

    #         diffs[k, :], _, _ = matcher.match(qfeat, dbfeat)

    #     cand_sorted = np.argsort(diffs)
    #     diffs = normalise_func(diffs, len(patch_sizes), patch_weights)
    #     reordered_preds.append(pred[cand_sorted])
    
    return reordered_preds

In [25]:
# val_predictions_subset = TensorDataset(*val_predictions[0:10])

In [37]:
# reranked_val_predictions = local_matcher_online(val_predictions, patchnetvlad_config, device)
reranked_val_predictions = local_matcher(val_predictions, patchnetvlad_config, device)

Processing query 1/2759Processing query 2/2759
Processing query 3/2759
Processing query 4/2759
Processing query 5/2759
Processing query 6/2759
Processing query 7/2759
Processing query 8/2759
Processing query 9/2759
Processing query 10/2759

Processing query 11/2759Processing query 12/2759

Processing query 13/2759
Processing query 14/2759
Processing query 15/2759
Processing query 16/2759
Processing query 17/2759
Processing query 18/2759
Processing query 19/2759Processing query 20/2759

Processing query 21/2759
Processing query 22/2759
Processing query 23/2759
Processing query 24/2759
Processing query 25/2759
Processing query 26/2759
Processing query 27/2759
Processing query 28/2759
Processing query 29/2759
Processing query 30/2759
Processing query 31/2759
Processing query 32/2759
Processing query 33/2759
Processing query 34/2759
Processing query 35/2759
Processing query 36/2759
Processing query 37/2759
Processing query 38/2759
Processing query 39/2759
Processing query 40/2759
Processin

In [35]:
def compute_recall(gt, predictions, numQ, n_values, recall_str=''):
    correct_at_n = np.zeros(len(n_values))
    for qIx, pred in enumerate(predictions):
        for i, n in enumerate(n_values):
            # if in top N then also in top NN, where NN > N
            if np.any(np.in1d(pred[:n], gt[qIx])):
                correct_at_n[i:] += 1
                break
    recall_at_n = correct_at_n.astype(np.float32) / numQ
    all_recalls = {}  # make dict for output
    for i, n in enumerate(n_values):
        all_recalls[n] = recall_at_n[i]
        print("====> Recall {}@{}: {:.4f}".format(recall_str, n, recall_at_n[i]))
    return all_recalls

In [38]:
n_values = [1, 5, 10, 20, 50, 100]
dataset_size = 27592
dataset_tolerance = 10

def get_positives(gt, dataset_tolerance, dataset_size):
    return [list(filter(lambda x: (x >= 0 and x < dataset_size), range(i.item() - dataset_tolerance, i.item() + dataset_tolerance + 1))) for i in gt]

gt_val = get_positives(val_predictions.tensors[1], dataset_tolerance, dataset_size)
global_recalls = compute_recall(gt_val, val_predictions.tensors[0], len(val_predictions), n_values, 'for Val EchoVPR')
local_recalls = compute_recall(gt_val, reranked_val_predictions, len(val_predictions), n_values, 'for Val EchoVPR+PatchNetVLAD')

====> Recall for Val EchoVPR@1: 0.4853
====> Recall for Val EchoVPR@5: 0.6049
====> Recall for Val EchoVPR@10: 0.6578
====> Recall for Val EchoVPR@20: 0.7148
====> Recall for Val EchoVPR@50: 0.7764
====> Recall for Val EchoVPR@100: 0.8173
====> Recall for Val EchoVPR+PatchNetVLAD@1: 0.6517
====> Recall for Val EchoVPR+PatchNetVLAD@5: 0.7398
====> Recall for Val EchoVPR+PatchNetVLAD@10: 0.7691
====> Recall for Val EchoVPR+PatchNetVLAD@20: 0.7887
====> Recall for Val EchoVPR+PatchNetVLAD@50: 0.8079
====> Recall for Val EchoVPR+PatchNetVLAD@100: 0.8173


In [29]:
reranked_test_predictions = local_matcher(test_predictions, patchnetvlad_config, device)

Processing query 1/24833
Processing query 2/24833Processing query 3/24833
Processing query 4/24833
Processing query 5/24833
Processing query 6/24833
Processing query 7/24833
Processing query 8/24833
Processing query 9/24833
Processing query 10/24833
Processing query 11/24833

Processing query 12/24833
Processing query 13/24833
Processing query 14/24833
Processing query 15/24833
Processing query 16/24833
Processing query 17/24833
Processing query 18/24833
Processing query 19/24833
Processing query 20/24833
Processing query 21/24833
Processing query 22/24833
Processing query 23/24833
Processing query 24/24833
Processing query 25/24833
Processing query 26/24833Processing query 27/24833

Processing query 28/24833
Processing query 29/24833
Processing query 30/24833
Processing query 31/24833
Processing query 32/24833
Processing query 33/24833
Processing query 34/24833Processing query 35/24833

Processing query 36/24833
Processing query 37/24833
Processing query 38/24833
Processing query 39/2

In [36]:
dataset_size = 27592
dataset_tolerance = 10

def get_positives(gt, dataset_tolerance, dataset_size):
    return [list(filter(lambda x: (x >= 0 and x < dataset_size), range(i.item() - dataset_tolerance, i.item() + dataset_tolerance + 1))) for i in gt]

gt_test = get_positives(test_predictions.tensors[1], dataset_tolerance, dataset_size)
    
test_echovpr_recalls = compute_recall(gt_test, test_predictions.tensors[0], len(test_predictions), n_values, 'for Test EchoVPR')
test_echovpr_patchnetvlad_recalls = compute_recall(gt_test, reranked_test_predictions, len(test_predictions), n_values, 'for Test EchoVPR+PatchNetVLAD')

====> Recall for Test EchoVPR@1: 0.4749
====> Recall for Test EchoVPR@5: 0.6097
====> Recall for Test EchoVPR@10: 0.6637
====> Recall for Test EchoVPR@20: 0.7140
====> Recall for Test EchoVPR@50: 0.7801
====> Recall for Test EchoVPR@100: 0.8275
====> Recall for Test EchoVPR+PatchNetVLAD@1: 0.6640
====> Recall for Test EchoVPR+PatchNetVLAD@5: 0.7598
====> Recall for Test EchoVPR+PatchNetVLAD@10: 0.7865
====> Recall for Test EchoVPR+PatchNetVLAD@20: 0.8061
====> Recall for Test EchoVPR+PatchNetVLAD@50: 0.8215
====> Recall for Test EchoVPR+PatchNetVLAD@100: 0.8275
